Doubly Linked Lists

Each node now has two pointers. In addition to the next pointer, we have a prev pointer which points to the previous node. If the prev pointer points to null, it is an indication that we are at the head of the linked list.

Operations of a Doubly Linked Lists

Insertion End

Similar to the singly linked list, adding a node to a doubly linked list will run O(1) time. Only this time, we have to update the prev pointer as well.

For example, if we have three nodes in our linked list, ListNode1, ListNode2 and ListNode3. Now we have another node, ListNode4, that we wish to insert at the end. We will have to update both the next pointer of ListNode3 and the prev pointer of ListNode4.

tail.next = ListNode4

ListNode4.prev = tail

tail = tail.next


Deletion End: 

Deleting at the end is also a O(1) operation.

1. First we get a reference to the node before the tail.
2. We update the next pointer of the node before the tail to null.
3. We update the tail to be the node before the tail.
4. (Optional) We can also update the prev pointer of the old tail to null.

ListNode2 = tail.prev

ListNode2.next = null

tail = ListNode2


Since we can insert and remove from the end in O(1) time, in theory, we could implement a stack with a linked list instead of an array. This is less common, but it is a possibility.

Access

Similar to singly linked lists, we cannot randomly access a node. So in the worst case, we will have to traverse n nodes before reaching the desired node. This would run in O(n) time.

Doubly linked lists have the benefit that we can traverse the list in both directions, as opposed to singly linked lists.

Time Complexity: 

Operation | Big - O Time | Notes
Access    | O(n)         | 
Search    | O(n)         |
Insertion | O(1)*        | Assuming you already have a reference to the node at the desired position
Deletion  | O(1)*        | Assuming you already have a reference to the node at the desired position

In [ ]:
class Node: 
    def __init__(self, key, val):
        self.key = key
        self.val = val
        # each node has 2 pointers 
        self.prev = self.next = None 

class LRUCache:  
    def __init__(self, capacity: int):
        self.capacity = capacity 
        self.cache = {} # map key to node, value is pointing to the Node
        # using a HM so we can quickly know what the value is given the key O(1)
        # the size of the HM does not exceed capacity.  

        # dummy nodes: tell us what are most recent and least recent. Initilize to zero.
        self.left = Node(0, 0) # left pointer = LRU,
        self.right = Node(0, 0) # right pointer = most recent 
        self.left.next = self.right 
        self.right.prev = self.left

    # remove node from left position, Least recent used
    def remove(self, node):
        prev, nxt = node.prev, node.next
        prev.next, nxt.prev = nxt, prev
    
    # insert node at right position, Most recent used
    def insert(self, node):
        prev, nxt = self.right.prev, self.right 
        prev.next = nxt.prev = node 
        node.prev, node.next = prev, nxt

    def get(self, key:int) -> int:
        if key in self.cache: 
            # remove and insert helper functions to update most recent linked list
            self.remove(self.cache[key])
            self.insert(self.cache[key])
            return self.cache[key].val 
        return -1 

    # if we have a key, thats already in our cache, a node already exists in our list 
    # with that same key value. We want to remove from our list
    def put(self, key:int, value: int) -> None:
        if key in self.cache:
            self.remove(self.cache[key]) # this is to remove node from left position, LRU
        self.cache[key] = Node(key, value) 
        self.insert(self.cache[key]) # insert into right position, most recent used

        if len(self.cache) > self.capacity:
            lru = self.left.next 
            self.remove(lru)
            del self.cache[lru.key]

Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]

Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4

- Key: the integer cache key (e.g., 1, 2, 3)
- Value: a pointer to the Node in the doubly linked list

So when you do self.cache[key], you get back the actual Node object, which lets you directly manipulate its position in the linked list in O(1) — no traversal needed.

Why store the Node and not just the value?

Without the hashmap storing Node references, if you needed to move a node to the "most recent" position, you'd have to traverse the entire linked list to find it — O(n). The hashmap gives you O(1) direct access to any node's memory address, so you can immediately call remove() and insert() on it.

Why does Node store key too?

Notice this line in put:
pythonlru = self.left.next   # get LRU node from linked list
del self.cache[lru.key]  # need the key to delete from hashmap!

You're traversing from the linked list back to the hashmap, so the node needs to carry its key with it. Without node.key, you'd have no way to clean up the hashmap entry when evicting the LRU.

So the two data structures work together:

- Hashmap: O(1) lookup by key → gives you the Node
- Linked list: maintains insertion order → tells you which Node to evict
- Node.key: lets you go in reverse (list → hashmap) for cleanup